In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

def get_output_layers(net):
    layer_names = net.getLayerNames()
    try:
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    except:
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

def draw_prediction(img, classes, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x, y), (x_plus_w, y_plus_h), color, 2)
    cv2.putText(img, label, (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def detect_and_display(image_path, classes, config_path, weights_path):
    # Read input image
    image = cv2.imread(image_path)
    Width, Height = image.shape[1], image.shape[0]
    scale = 0.00392

    # Load YOLOv4
    net = cv2.dnn.readNet(weights_path, config_path)

    # Prepare image for YOLO
    blob = cv2.dnn.blobFromImage(image, scale, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Forward pass and get output layers
    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4

    # Process detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold and class_id == person_class_index:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w // 2
                y = center_y - h // 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
                
    # Apply non-maximum suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # Count total persons detected
    total_persons = len(indices)

    # Display total persons detected
    persons_label.config(text=f"Total persons detected: {total_persons}")

    # Display the output image with bounding boxes around persons
    image_with_boxes = image.copy()
    for i in indices:
        box = boxes[i]
        x, y, w, h = box
        class_id = class_ids[i]
        confidence = confidences[i]
        draw_prediction(image_with_boxes, classes, class_id, confidence, round(x), round(y), round(x + w), round(y + h))

    # Convert image to PIL format and resize
    image_with_boxes = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB)
    image_with_boxes = Image.fromarray(image_with_boxes)
    image_with_boxes = image_with_boxes.resize((250, 250))  # Resize detected image
    image_with_boxes = ImageTk.PhotoImage(image_with_boxes)

    # Update the displayed image
    detected_image_label.config(image=image_with_boxes)
    detected_image_label.image = image_with_boxes

    # Resize uploaded image and display
    uploaded_image = Image.open(image_path)
    uploaded_image = uploaded_image.resize((250, 250))  # Resize uploaded image
    uploaded_image = ImageTk.PhotoImage(uploaded_image)
    uploaded_image_label.config(image=uploaded_image)
    uploaded_image_label.image = uploaded_image

def browse_image():
    filename = filedialog.askopenfilename()
    if filename:
        detect_and_display(filename, classes, config_path, weights_path)

root = tk.Tk()
root.title("Person Detection")
root.geometry("1200x700")  # Increased height to accommodate the headings

# Load background image
background_image_path = 'pic6.jpg'
background_image = Image.open(background_image_path)
background_image = ImageTk.PhotoImage(background_image.resize((1200, 700)))  # Resize background image to match root window size

background_label = tk.Label(root, image=background_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)

# Define your YOLOv4 file paths here
class_path = 'coco.names'
config_path = 'yolov4.cfg'
weights_path = 'yolov4.weights'

# Load classes
with open(class_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Filter classes to include only 'person'
person_class_index = classes.index('person')

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# Browse image button
browse_button = tk.Button(root, text="Browse Image", command=browse_image, bg="#008CBA", fg="white", font=("Helvetica", 14))
browse_button.place(relx=0.5, rely=0.1, anchor=tk.CENTER)

# Uploaded image heading
uploaded_image_heading = tk.Label(root, text="Uploaded Image", font=("Helvetica", 14))
uploaded_image_heading.place(relx=0.2, rely=0.3, anchor=tk.CENTER)

# Detected image heading
detected_image_heading = tk.Label(root, text="Detected Image", font=("Helvetica", 14))
detected_image_heading.place(relx=0.75, rely=0.3, anchor=tk.CENTER)

# Uploaded image label
uploaded_image_label = tk.Label(root)
uploaded_image_label.place(relx=0.2, rely=0.6, anchor=tk.CENTER)

# Detected image label
detected_image_label = tk.Label(root)
detected_image_label.place(relx=0.75, rely=0.6, anchor=tk.CENTER)

# Total persons detected label
persons_label = tk.Label(root, text="Total persons detected: 0", font=("Helvetica", 16))
persons_label.place(relx=0.5, rely=0.9, anchor=tk.CENTER)

root.mainloop()
